In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Breast_Cancer"
cohort = "GSE225328"

# Input paths
in_trait_dir = "../../input/GEO/Breast_Cancer"
in_cohort_dir = "../../input/GEO/Breast_Cancer/GSE225328"

# Output paths
out_data_file = "../../output/preprocess/Breast_Cancer/GSE225328.csv"
out_gene_data_file = "../../output/preprocess/Breast_Cancer/gene_data/GSE225328.csv"
out_clinical_data_file = "../../output/preprocess/Breast_Cancer/clinical_data/GSE225328.csv"
json_path = "../../output/preprocess/Breast_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome profiling in early-stage luminal breast cancer"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease: early-stage luminal breast cancer'], 1: ['Sex: female']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# According to the background information, this is a transcriptome profiling study
# which typically means gene expression data is available
is_gene_available = True

# 2.1 Data Availability
# Looking at the Sample Characteristics Dictionary:
# Key 0 has "disease: early-stage luminal breast cancer" which is related to the trait (Breast Cancer)
# Key 1 has "Sex: female" which is gender information
# There is no age information available

trait_row = 0  # Disease information is in row 0
age_row = None  # Age information is not available
gender_row = 1  # Gender information is in row 1

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait values to binary format."""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Since all samples are "early-stage luminal breast cancer", 
    # all will be converted to 1 (indicating presence of breast cancer)
    if "breast cancer" in value.lower():
        return 1
    else:
        return None  # For any unexpected values

def convert_age(value):
    """Convert age values to continuous format."""
    # Age data is not available, but we include this function for completeness
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender values to binary format (0 for female, 1 for male)."""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering and saving metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of clinical features:")
    print(preview_df(clinical_features_df))
    
    # Save the clinical features as a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM7043537': [1.0, 0.0], 'GSM7043538': [1.0, 0.0], 'GSM7043539': [1.0, 0.0], 'GSM7043540': [1.0, 0.0], 'GSM7043541': [1.0, 0.0], 'GSM7043542': [1.0, 0.0], 'GSM7043543': [1.0, 0.0], 'GSM7043544': [1.0, 0.0], 'GSM7043545': [1.0, 0.0], 'GSM7043546': [1.0, 0.0], 'GSM7043547': [1.0, 0.0], 'GSM7043548': [1.0, 0.0], 'GSM7043549': [1.0, 0.0], 'GSM7043550': [1.0, 0.0], 'GSM7043551': [1.0, 0.0], 'GSM7043552': [1.0, 0.0], 'GSM7043553': [1.0, 0.0], 'GSM7043554': [1.0, 0.0], 'GSM7043555': [1.0, 0.0], 'GSM7043556': [1.0, 0.0], 'GSM7043557': [1.0, 0.0], 'GSM7043558': [1.0, 0.0], 'GSM7043559': [1.0, 0.0], 'GSM7043560': [1.0, 0.0], 'GSM7043561': [1.0, 0.0], 'GSM7043562': [1.0, 0.0], 'GSM7043563': [1.0, 0.0], 'GSM7043564': [1.0, 0.0], 'GSM7043565': [1.0, 0.0], 'GSM7043566': [1.0, 0.0], 'GSM7043567': [1.0, 0.0], 'GSM7043568': [1.0, 0.0], 'GSM7043569': [1.0, 0.0], 'GSM7043570': [1.0, 0.0], 'GSM7043571': [1.0, 0.0], 'GSM7043572': [1.0, 0.0], 'GSM7043573': [1.0, 0.0], 'GSM704

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Breast_Cancer/GSE225328/GSE225328_family.soft.gz
Matrix file: ../../input/GEO/Breast_Cancer/GSE225328/GSE225328-GPL18402_series_matrix.txt.gz
Found the matrix table marker at line 60
Gene data shape: (2006, 125)
First 20 gene/probe identifiers:
['hsa-let-7a-3p', 'hsa-let-7a-5p', 'hsa-let-7b-3p', 'hsa-let-7b-5p', 'hsa-let-7c', 'hsa-let-7d-3p', 'hsa-let-7d-5p', 'hsa-let-7e-3p', 'hsa-let-7e-5p', 'hsa-let-7f-1-3p', 'hsa-let-7f-2-3p', 'hsa-let-7f-5p', 'hsa-let-7g-3p', 'hsa-let-7g-5p', 'hsa-let-7i-3p', 'hsa-let-7i-5p', 'hsa-miR-1', 'hsa-miR-100-3p', 'hsa-miR-100-5p', 'hsa-miR-101-3p']


### Step 4: Gene Identifier Review

In [5]:
# Based on the output from the previous step, I can see that the gene identifiers
# are miRNA identifiers (e.g., "hsa-let-7a-3p", "hsa-miR-1", etc.)
# These are proper standard miRNA names for human miRNAs (hsa prefix = Homo sapiens)
# They are not gene symbols (like BRCA1, TP53) and would need to be mapped if we want
# to convert to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Looking at the output, it appears the gene symbols are in the 'ORF' column
# and the probe IDs are in the 'ID' column
print("\nExamining ID and ORF columns format (first 3 rows):")
if 'ID' in gene_annotation.columns and 'ORF' in gene_annotation.columns:
    for i in range(min(3, len(gene_annotation))):
        print(f"Row {i}: ID={gene_annotation['ID'].iloc[i]}, ORF={gene_annotation['ORF'].iloc[i]}")

    # Check the quality and completeness of the mapping
    non_null_symbols = gene_annotation['ORF'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nORF column completeness: {non_null_symbols}/{total_rows} rows ({non_null_symbols/total_rows:.2%})")



Gene annotation preview:
Columns in gene annotation: ['ID', 'miRNA_ID', 'ACCESSION_STRING', 'CONTROL_TYPE', 'SPOT_ID', 'SPOT_ID.1']
{'ID': ['hsa-let-7a-3p', 'hsa-let-7a-5p', 'hsa-let-7b-3p'], 'miRNA_ID': ['hsa-let-7a-3p', 'hsa-let-7a-5p', 'hsa-let-7b-3p'], 'ACCESSION_STRING': ['mir|hsa-let-7a-3p|mir|MIMAT0004481|mir|hsa-let-7a*_v17.0|mir|MIMAT0004481', 'mir|hsa-let-7a-5p|mir|MIMAT0000062|mir|hsa-let-7a_v17.0|mir|MIMAT0000062', 'mir|hsa-let-7b-3p|mir|MIMAT0004482|mir|hsa-let-7b*_v17.0|mir|MIMAT0004482'], 'CONTROL_TYPE': [False, False, False], 'SPOT_ID': [nan, nan, nan], 'SPOT_ID.1': [nan, nan, nan]}

Examining ID and ORF columns format (first 3 rows):


### Step 6: Gene Identifier Mapping

In [7]:
# Based on the exploration of columns, we can confirm this is miRNA data, not gene expression data
# The identifiers are miRNA IDs (e.g., hsa-let-7a-3p) which don't map to standard gene symbols

# Let's examine a sample of the gene expression data and annotation to confirm
gene_expression_data = get_genetic_data(matrix_file)
print("\nGene expression data sample (first 5 rows, 3 columns):")
sample_cols = gene_expression_data.columns[:3].tolist()
print(gene_expression_data.iloc[:5, :3])

# Update our gene availability flag since this isn't standard gene expression data
is_gene_available = False
print("\nThis dataset contains miRNA expression data, not standard gene expression data.")
print("miRNAs are small non-coding RNAs that regulate gene expression but are not genes themselves.")
print("For the purpose of gene-trait association studies, we require standard gene expression data.")

# Save the updated metadata to reflect that this dataset isn't suitable
is_trait_available = True  # We confirmed trait data is available in earlier steps
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    note="Dataset contains miRNA expression data instead of gene expression data."
)

print("\nDataset marked as not containing suitable gene expression data for our analysis.")


Gene expression data sample (first 5 rows, 3 columns):
               GSM7043537  GSM7043538  GSM7043539
ID                                               
hsa-let-7a-3p   -2.842319    0.656993   -3.119569
hsa-let-7a-5p   12.236782   11.668952   12.020029
hsa-let-7b-3p    3.586698   -3.123689    0.167606
hsa-let-7b-5p   12.979993   12.667322   12.236782
hsa-let-7c      11.518827   10.673440   10.153807

This dataset contains miRNA expression data, not standard gene expression data.
miRNAs are small non-coding RNAs that regulate gene expression but are not genes themselves.
For the purpose of gene-trait association studies, we require standard gene expression data.

Dataset marked as not containing suitable gene expression data for our analysis.
